In [ ]:
!pip install sentence_transformers

트랜스포머를 이용한 한국어 챗봇 실습에서 사용했던 데이터를 그대로 사용합니다.

In [2]:
import urllib.request
import pandas as pd

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


문장 임베딩을 얻기 위해서 사전 훈련된 BERT를 로드합니다. 여기서는 한국어도 포함되어 학습된 다국어 모델을 로드합니다.

In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

모델의 이름은 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'인데 이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미입니다. 다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.

SentenceTransformer로 로드할 수 있는 다양한 모델에 대한 리스트는 아래의 링크에서 확인 가능합니다.
해당 링크에 한국어 버전의 모델들 또한 공개되어 있으니 방문해보세요.

링크 : https://huggingface.co/models?library=sentence-transformers

데이터에서 모든 질문열. 즉, train_data['Q']에 대해서 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장합니다.

In [7]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [8]:
train_data

,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0,"[0.2017957, -0.034438018, 1.5395725, 0.0106974..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.07716595, -0.03427819, 0.8624428, 0.0263606..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.104452446, -0.012432282, 1.0132879, 0.02250..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.09760743, -0.046716843, 0.89369434, 0.02104..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002873, 0.031961218, 1.4915432, 4.33528e..."
...,...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[0.15363301, -0.32605588, 0.69876784, 0.109140..."
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2,"[0.15363301, -0.32605588, 0.69876784, 0.109140..."
11820,흑기사 해주는 짝남.,설렜겠어요.,2,"[-0.036085192, -0.00526488, 0.8913534, -0.0117..."
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2,"[-0.08468659, -0.14302917, 0.11526107, 0.38679..."


In [9]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [12]:
A = np.array([2,3])
print(norm(A))
B = np.array([3,1])
print(norm(B))
print(dot(A, B)/(norm(A)*norm(B)))

3.605551275463989
3.1622776601683795
0.7893522173763263


In [34]:
A = np.array([3,0])
print(norm(A))
B = np.array([0,3])
print(norm(B))
print(dot(A, B)/(norm(A)*norm(B)))

3.0
3.0
0.0


In [35]:
A = np.array([-3,0])
print(norm(A))
B = np.array([3,0])
print(norm(B))
print(dot(A, B)/(norm(A)*norm(B)))

3.0
3.0
-1.0


In [33]:
A = np.array([1,1,1,1,0,1])
print(norm(A))
B = np.array([2,2,2,2,0,2])
print(norm(B))
print(dot(A, B)/(norm(A)*norm(B)))

2.23606797749979
4.47213595499958
0.9999999999999998


두 개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim를 정의합니다.

In [13]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [29]:
embedding = model.encode('결혼하고싶어')
print(embedding.shape)

(768,)


return_answer 함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열. 즉, train_data['embedding']에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냅니다. 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴합니다.

In [14]:
def return_similar_answer(input):
    embedding = model.encode(input)
    
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

이제 챗봇을 테스트해봅시다.

In [15]:
return_similar_answer('결혼하고싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [16]:
return_similar_answer('나랑 커피먹을래?')

'카페인이 필요한 시간인가 봐요.'

In [17]:
return_similar_answer('반가워')

'저도 반가워요.'

In [18]:
return_similar_answer('사랑해')

'상대방에게 전해보세요.'

In [19]:
return_similar_answer('너는 누구니?')

'저는 위로봇입니다.'

In [20]:
return_similar_answer('영화')

'저도 영화 보여주세요.'

In [21]:
return_similar_answer('너무 짜증나')

'짜증날 땐 짜장면'

In [22]:
return_similar_answer('화가납니다')

'화를 참는 연습을 해보세요.'

In [23]:
return_similar_answer('나랑 놀자')

'지금 그러고 있어요.'

In [24]:
return_similar_answer('나랑 게임하자')

'같이 놀아요.'

In [25]:
return_similar_answer('출근하기싫어')

'씻고 푹 쉬세요.'

In [26]:
return_similar_answer('여행가고싶다')

'이김에 떠나보세요.'

In [27]:
return_similar_answer('너 말 잘한다')

'그런 사람이 있으면 저 좀 소개시켜주세요.'